In [1]:
import os

colab = 'COLAB_GPU' in os.environ
if colab:
    print("Running in Google Colab")
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    DRIVE_FOLDER = '/content/drive/MyDrive/Data/BikePricePredict/'

Running in Google Colab
Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Data/BikePricePredict/bicycles_all_keep_nodupes3_resized.zip -d /content/data/

Archive:  /content/drive/MyDrive/Data/BikePricePredict/bicycles_all_keep_nodupes3_resized.zip
replace /content/data/bicycles_all_keep_nodupes3_resized/00004c6e-c371-330a-936b-7e1560048f66_375.50.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

In [ ]:
def get_dataset_path_and_labels(dataset_path):
    labels = []
    files = []
    for file in os.listdir(dataset_path):
        if 'Steuern' in file:
          continue
        label = file.split('_')[-1].replace('.jpg','').replace('ÿ','').replace('ÿ','').replace('ab','').strip()
        labels.append(float(label))
        files.append(os.path.join(dataset_path, file))
    return files, labels

In [ ]:

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)

    ''' # preprocess
    image = tf.image.resize(image, [256, 256])
    image /= 255.0 # normalize '''


    '''  # Flip the image horizontally
    flipped_image = tf.image.flip_left_right(image) '''

   
    
    # Add the augmented images to the dataset
    ''' images = tf.stack([image, flipped_image])
    labels = tf.stack([label, label, label]) '''

    return image, label





In [ ]:
from tensorflow.keras import layers
import numpy as np

def get_dataset(dataset_path, skip, take, random_seed):
    
    files, labels = get_dataset_path_and_labels(dataset_path)

    shuffled_index = np.random.RandomState(seed=random_seed).permutation(len(files))
    take_index = shuffled_index[skip:skip+take]
    files = [files[i] for i in take_index]
    labels = [labels[i] for i in take_index]

    ds = tf.data.Dataset.from_tensor_slices((files, labels))
    ds = ds.map(load_and_preprocess_image)
    return ds


import os

if colab:
    #dir = DRIVE_FOLDER+'bicycles_resized_1024_20k'
    dir = '/content/data/bicycles_all_keep_nodupes3_resized/'
  
else:
    print("Running locally")


IMG_SIZE = 256
data_preprocessing = tf.keras.Sequential([
  layers.Resizing(IMG_SIZE, IMG_SIZE),
  layers.Rescaling(1./255),
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.01),
  
])




test_size = 1000
dev_size = 1000


seed = 4
# no shuffling
test_dataset = get_dataset(dir, skip=0, take=test_size, random_seed=seed)
dev_dataset = get_dataset(dir, skip=test_size, take=dev_size, random_seed=seed)
train_dataset = get_dataset(dir, skip=test_size+dev_size, take=10000000, random_seed=seed)




train_dataset = train_dataset.map(lambda x, y : (data_preprocessing(x, training=True), y) )
dev_dataset = dev_dataset.map(lambda x, y : (data_preprocessing(x, training=False), y) )
test_dataset = test_dataset.map(lambda x, y : (data_preprocessing(x, training=False), y) )

In [ ]:
len(train_dataset), len(dev_dataset), len(test_dataset)

In [ ]:
x, y = next(train_dataset.take(1).as_numpy_iterator())

In [ ]:
import PIL
# view x as image with PIL
PIL.Image.fromarray((x * 255).astype(np.uint8))

In [ ]:
model2 = tf.keras.applications.ResNet50V2(weights='imagenet', include_top=False)
model2.summary()

In [ ]:
# count conv2d layers in model2
conv2d_indices = []
for i, layer in enumerate(model2.layers):
    if layer.name.endswith("conv"):
        conv2d_indices.append(i)

len(conv2d_indices)

In [ ]:
53 * 0.25

In [ ]:
 # each block has approximately 3 conv2d layers
idx  = -3*10 +1
model2.layers[conv2d_indices[idx]].name, conv2d_indices[idx]

In [ ]:
def save_weights(model, weights_path):
  for layer in model.layers:
    layer.trainable=False
  ''' for layer in base_model.layers:
    layer.trainable=False '''
  
  model.save(weights_path)

In [ ]:


def make_model(base_model, unfreeze_start:int, weights_path=None):
  
  
    
    

  base_model.trainable = False

  inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
  x = base_model(inputs, training=True) # training=True only enabled for unfrozen layers (?)
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  ''' x = tf.keras.layers.Dense(256, activation='relu')(x) '''
  '''   x = tf.keras.layers.Dense(128, activation='relu')(x)
  x = tf.keras.layers.Dense(64, activation='relu')(x) '''
  outputs = tf.keras.layers.Dense(1, activation='relu')(x) # use relu because prices are always positive
  model = tf.keras.Model(inputs, outputs)

 
  if weights_path is not None:
    unfr = int(weights_path.split('unfr')[-1].split('_')[0])
    assert unfreeze_start == unfr

    model.load_weights(weights_path)

  # freeze base model
  for layer in base_model.layers[unfreeze_start:]: # last, conv5 section
    layer.trainable = True 

  
  return model
  
# create keras resnet 50 v2 model
base_model = tf.keras.applications.ResNet50V2(include_top=False, weights='imagenet')
unfreeze_start = 163



In [ ]:

#model = make_model(base_model, unfreeze_start, '/content/drive/MyDrive/Data/BikePricePredict/weights_unfr163_bs32_lr0.0001_ep200.h5')
model = make_model(base_model, unfreeze_start)

In [ ]:
model = make_model(base_model, unfreeze_start, DRIVE_FOLDER+'weights_unfr163_bs32_lr0.001_28k_1k_1k_256MAE_ep55.h5') 


In [ ]:
# save_weights(model, '/content/drive/MyDrive/Data/BikePricePredict/test.h5')

In [ ]:
# model = make_model(base_model, '/content/drive/MyDrive/Data/BikePricePredict/test.h5')

In [ ]:
import matplotlib.pyplot as plt
x,y = next(train_dataset.take(3000).batch(3000).as_numpy_iterator())

In [ ]:
plt.hist(y)
plt.title('bike prices')

In [ ]:
import pandas as pd
def fit(model, dataset, dev_dataset, batch_size , epochs, save_after_epochs, start_epoch_num, lr, unfreeze_start,  custom_tag='', dir=DRIVE_FOLDER):
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr, ), # beta1, beta2, weight_decay
                loss=tf.keras.losses.MeanSquaredError(),
                metrics=['mse', 'mae']
              )
  for ep in range(0, epochs, save_after_epochs):
    print(f'fit {ep+start_epoch_num}')
    histories = None
    with tf.device('/GPU:0'):
      for _ in range(save_after_epochs):
        history = model.fit(dataset.batch(batch_size), epochs=1)
        loss, mse, mae = model.evaluate(dev_dataset.batch(batch_size))
        print(f'dev mse: {mse}, mae: {mae}')
        if histories is None:
          histories = history.history
          histories['dev_mse'] = [mse]
          histories['dev_mae'] = [mae]
        else:
          for k, v in history.history.items():
            histories[k] += v
          histories['dev_mse'].append(mse)
          histories['dev_mae'].append(mae)

    # Convert the history to a Pandas DataFrame
    history_df = pd.DataFrame(histories)
    print(history_df)
    # Save the history to a CSV file
    history_df.to_csv(dir+f'history_unfr{str(unfreeze_start)}_bs{str(batch_size)}_lr{str(lr)}_{custom_tag}_ep{str(ep+start_epoch_num)}.csv', index=False)
    save_weights(model, dir+f'weights_unfr{str(unfreeze_start)}_bs{str(batch_size)}_lr{str(lr)}_{custom_tag}_ep{str(ep+start_epoch_num)}.h5')
    

In [ ]:
fit(model, dataset=train_dataset, dev_dataset=dev_dataset, batch_size=32, epochs=100000, save_after_epochs=5, start_epoch_num=15, lr=0.001, unfreeze_start = unfreeze_start, custom_tag='28k_1k_1k_256MAE')

In [ ]:
np.sqrt(5695112) 

In [ ]:
x,y = next(test_dataset.take(10).batch(10).as_numpy_iterator())
model.predict(x), y

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def plot_loss(unfrozen_layers, lr:str, batch_size=None, minus_epochstart=0, custom_tag=None):
  y = []
  x = []
  for file in os.listdir(DRIVE_FOLDER):
    if 'unfr'+str(unfrozen_layers) in file and 'lr'+str(lr) in file and 'history' in file:
      if batch_size is not None:
        if 'bs'+str(batch_size) not in file:
          continue
      if custom_tag is not None:
        if custom_tag not in file:
          continue

      epochstart = file.split('ep')[-1].replace('.csv','')
      losses = pd.read_csv(DRIVE_FOLDER+file)['loss'].tolist() 
      xi = list(np.arange(len(losses)) +int(epochstart) - minus_epochstart)
      y += losses
      x += xi
  #plt.title(f'RESNET50 v2')
  
  label = f'lr:{lr}, bs:{batch_size}, unfr.lay.{unfrozen_layers},  {custom_tag if custom_tag is not None else ""}'
  plt.scatter(x, y, s = np.ones(len(y)), label=label)
  plt.legend()


  





In [ ]:
(
plot_loss(163, '0.001', batch_size=32),
''' plot_loss(88, '0.001', batch_size=32), 
plot_loss(163, '0.0001', batch_size=32),
plot_loss(163, '0.001', batch_size=128),  
plot_loss(163, '0.003', minus_epochstart=100, batch_size=128), 
 ''',


plot_loss(163, '0.001', batch_size=32, custom_tag='moredata'),
plot_loss(163, '0.001', batch_size=32, custom_tag='28k_1k_1k_256MAE'),

)
plt.savefig(DRIVE_FOLDER+'effect_of_more_data28k.png')

In [ ]:
# best lr 0.001, best bs 32 (128, 32)
# 88 vs 163 (first pos of unfreeze) does not make a big difference

In [ ]:



# train model, specify batch size
batch_size = 128
epochs = 50
if colab:
  with tf.device('/GPU:0'):
      history = model.fit(dataset.batch(batch_size), epochs=epochs)
else:
  history = model.fit(dataset.batch(batch_size), epochs=epochs)

# plot history
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])

In [ ]:
# visualize part of the image which contributed most to the prediction
def visualize_heatmap(model, image):
    # get prediction
    prediction = model.predict(image[np.newaxis, ...])[0]
    # get gradient
    with tf.GradientTape() as tape:
        # cast image to float32
        inputs = tf.cast(image[np.newaxis, ...], tf.float32)
        # watch the input pixels
        tape.watch(inputs)
        # get the predictions
        predictions = model(inputs)
      
    print(predictions.shape)
    # get the gradient of the loss with respect to the inputs
    gradients = tape.gradient(predictions, inputs)
    # get the mean gradient across the channels
    gradient = tf.reduce_mean(gradients, axis=(0, 3))
    # get the mean prediction across the channels
    prediction = predictions#tf.reduce_mean(predictions, axis=3)[0]
    # scale the prediction
    prediction /= tf.reduce_max(prediction)
    # scale the gradient
    gradient /= tf.reduce_max(gradient)
    # upsample the prediction and the gradient
    prediction = tf.image.resize(prediction, [256, 256])
    gradient = tf.image.resize(gradient, [256, 256])
    # convert to numpy
    prediction = prediction.numpy()
    gradient = gradient.numpy()
    # plot the prediction and the gradient
    plt.figure(figsize=(10, 10))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.imshow(prediction, cmap='jet', alpha=0.5)
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.imshow(image)
    plt.imshow(gradient, cmap='jet', alpha=0.5)
    plt.axis('off')
    plt.show()

visualize_heatmap(model, x)